In [22]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

In [23]:
train=pd.read_csv('dataset/train.txt', sep=';')
train.columns=['Text','Emotions']
train.head()

,Text,Emotions
0,i can go from feeling so hopeless to so damned...,sadness
1,im grabbing a minute to post i feel greedy wrong,anger
2,i am ever feeling nostalgic about the fireplac...,love
3,i am feeling grouchy,anger
4,ive been feeling a little burdened lately wasn...,sadness


In [24]:
text=train['Text'].tolist()
labels=train['Emotions'].tolist()

In [25]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(text)

In [26]:
sequences=tokenizer.texts_to_sequences(text)
max_len=max([len(seq) for seq in sequences])
padded_sequences=pad_sequences(sequences,max_len)

In [27]:
label_encoder=LabelEncoder()
labels=label_encoder.fit_transform(labels)

In [28]:
one_hot_encoding=keras.utils.to_categorical(labels)

In [32]:
xtrain,xtest,ytrain,ytest=train_test_split(padded_sequences, one_hot_encoding,test_size=0.2)

In [30]:
#Define the Model

model=Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1,
                   output_dim=128,input_length=max_len))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=len(one_hot_encoding[0]),activation='softmax'))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


In [33]:
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))

Epoch 1/10
450/450 [==============================] - 12s 11ms/step - loss: 1.2467 - accuracy: 0.5322 - val_loss: 0.7287 - val_accuracy: 0.7503
Epoch 2/10
450/450 [==============================] - 4s 10ms/step - loss: 0.2745 - accuracy: 0.9160 - val_loss: 0.5146 - val_accuracy: 0.8314
Epoch 3/10
450/450 [==============================] - 5s 10ms/step - loss: 0.0531 - accuracy: 0.9857 - val_loss: 0.5934 - val_accuracy: 0.8261
Epoch 4/10
450/450 [==============================] - 5s 10ms/step - loss: 0.0223 - accuracy: 0.9953 - val_loss: 0.6698 - val_accuracy: 0.8206
Epoch 5/10
450/450 [==============================] - 4s 10ms/step - loss: 0.0190 - accuracy: 0.9963 - val_loss: 0.6476 - val_accuracy: 0.8261
Epoch 6/10
450/450 [==============================] - 4s 10ms/step - loss: 0.0147 - accuracy: 0.9967 - val_loss: 0.6853 - val_accuracy: 0.8217
Epoch 7/10
450/450 [==============================] - 4s 10ms/step - loss: 0.0149 - accuracy: 0.9968 - val_loss: 0.6970 - val_accuracy: 0.824

In [40]:
def prediction(text):
    input_sequence = tokenizer.texts_to_sequences([text])
    padded_input_sequence = pad_sequences(input_sequence, max_len)
    prediction = model.predict(padded_input_sequence)
    predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
    return predicted_label

In [53]:
input_text='i left with my bouquet of red and yellow tulips under my arm feeling slightly more optimistic than when i arrived'
prediction(input_text)

1/1 [==============================] - 0s 36ms/step


array(['joy'], dtype='<U8')